In [ ]:
#| default_exp datasets

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from cfnet.import_essentials import *
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
import json
import os
from urllib.request import urlretrieve

In [ ]:
#| export
class Dataset:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        assert self.X.shape[0] == self.y.shape[0]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
#| export
# copy from https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html#data-loading-with-pytorch
def _numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [_numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class DataLoader:
    def __init__(self,
                 dataset: Dataset,          # dataset, a Dataset object
                 batch_size: int=1,         # batch size
                 shuffle: bool=False,       # if true, dataloader shuffles before sampling each batch
                 seed: int=42,              # seed for random number generator
                 sampler=None,
                 batch_sampler=None,
                 num_workers=0,
                 collate_fn=_numpy_collate, # collate function, default is _numpy_collate()
                 pin_memory=False,
                 drop_last: bool=False,     # if true, dataloader drops the last batch that is less than the batch size
                 timeout=0,
                 worker_init_fn=None
                 ):
        # Attributes from pytorch data loader (implemented)
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.collate_fn = collate_fn
        self.drop_last = drop_last

        # Attributes from pytorch data loader (not implemented)
        self.sampler = sampler
        self.batch_sampler = batch_sampler
        self.num_workers = num_workers
        self.pin_memory = pin_memory
        self.worker_init_fn = worker_init_fn
        self.timeout = timeout

        self.data_len: int = len(dataset)  # Length of the dataset
        self.key_seq: hk.PRNGSequence = hk.PRNGSequence(self.seed)  # random number sequence
        self.key_seq.reserve(len(self))  # generate some random number as key based on the number of batches
        self.key = next(self.key_seq)  # obtain a random key from the sequence
        self.indices: jax.numpy.array = jax.numpy.arange(self.data_len)  # available indices in the dataset
        self.pose: int = 0  # record the current position in the dataset

    def __len__(self):
        if self.drop_last:
            batches = len(self.dataset) // self.batch_size  # get the floor of division
        else:
            batches = -(len(self.dataset) // -self.batch_size)  # get the ceil of division
        return batches

    def __next__(self):
        if self.pose <= self.data_len:
            if self.shuffle:
                self.key = next(self.key_seq)
                self.indices = jax.random.permutation(self.key, self.indices)
            batch_data = [self.dataset[i] for i in self.indices[:self.batch_size]]
            self.indices = self.indices[self.batch_size:]
            if self.drop_last and len(self.indices)<self.batch_size:
                self.pose = 0
                self.indices = jax.numpy.arange(self.data_len)
                raise StopIteration
            self.pose += self.batch_size
            return self.collate_fn(batch_data)
        else:
            self.pose = 0
            self.indices = jax.numpy.arange(self.data_len)
            raise StopIteration

    def __iter__(self):
        return self


def find_imutable_idx_list(
    imutable_col_names: List[str],
    discrete_col_names: List[str],
    continuous_col_names: List[str],
    cat_arrays: List[List[str]]
) -> List[int]:
    imutable_idx_list = []
    for idx, col_name in enumerate(continuous_col_names):
        if col_name in imutable_col_names:
            imutable_idx_list.append(idx)

    cat_idx = len(continuous_col_names)

    for i, (col_name, cols) in enumerate(zip(discrete_col_names, cat_arrays)):
        cat_end_idx = cat_idx + len(cols)
        if col_name in imutable_col_names:
            imutable_idx_list += list(range(cat_idx, cat_end_idx))
        cat_idx = cat_end_idx
    return imutable_idx_list

In [ ]:
#| export
class DataModuleConfigs(BaseParser):
    batch_size: int
    discret_cols: List[str] = []
    continous_cols: List[str] = []
    imutable_cols: List[str] = []
    normalizer: Optional[Any] = None
    encoder: Optional[Any] = None
    sample_frac: Optional[float] = None

In [ ]:
#| export
class TabularDataModule:
    discret_cols: List[str] = []
    continous_cols: List[str] = []
    imutable_cols: List[str] = []
    normalizer: Optional[Any] = None
    encoder: Optional[OneHotEncoder] = None
    data: Optional[pd.DataFrame] = None
    sample_frac: Optional[float] = None
    batch_size: int = 128
    data_name: str = ""

    def __init__(self, data_configs:dict or str=None):
        if isinstance(data_configs, str):
            data_configs = self.name_to_config(data_configs)
            self.data = pd.read_csv(Path(data_configs['data_dir']))
        elif isinstance(data_configs, dict):
            # read data
            self.data = pd.read_csv(Path(data_configs['data_dir']))

        # update configs
        self._update_configs(data_configs)
        self.check_cols()
        # update cat_idx
        self.cat_idx = len(self.continous_cols)
        # prepare data
        self.prepare_data()

    def name_to_config(self, data_name):
        with open('../assets/configs/{}.json'.format(data_name)) as json_file:
            data = json.load(json_file)
            data_configs['data_name']=data_name
            data_configs['discret_cols']=data['discret_cols']
            data_configs['continous_cols']=data['continous_cols']
            data_configs['imutable_cols']=data.get('imutable_cols', [])
            data_configs['sample_frac']=data.get('sample_frac', [])
            data_configs['normalizer']=data.get('normalizer', [])
            data_configs['encoder']=data.get('encoder', [])
            data_configs['data_dir'] = self.download_data(data_name)
        return data_configs

    def download_data(self,data_name):
        url='https://github.com/BirkhoffG/cfnet/raw/master/assets/data/{}.csv'.format(data_name)
        path=os.getcwd()
        print(path)
        if not os.path.exists("cf_data"):
            os.makedirs("cf_data")
        path = os.path.join(path, "cf_data", "{}.csv".format(data_name))
        if os.path.exists(path):
            return path
        else:
            urlretrieve(url,path)
            return path

    def check_cols(self):
        self.data = self.data.astype({col: np.float for col in self.continous_cols})
        # check imutable cols
        cols = self.continous_cols + self.discret_cols
        for col in self.imutable_cols:
            assert col in cols, \
                f"imutable_cols=[{col}] is not specified in `continous_cols` or `discret_cols`."

    def _update_configs(self, configs):
        for k, v in configs.items():
            setattr(self, k, v)

    def prepare_data(self):
        def split_x_and_y(data: pd.DataFrame):
            X = data[data.columns[:-1]]
            y = data[[data.columns[-1]]]
            return X, y

        X, y = split_x_and_y(self.data)

        # preprocessing
        if self.normalizer:
            X_cont = self.normalizer.transform(X[self.continous_cols])
        else:
            self.normalizer = MinMaxScaler()
            X_cont = self.normalizer.fit_transform(
                X[self.continous_cols]) if self.continous_cols else np.array([[] for _ in range(len(X))])

        if self.encoder:
            X_cat = self.encoder.transform(X[self.discret_cols])
        else:
            self.encoder = OneHotEncoder(sparse=False)
            X_cat = self.encoder.fit_transform(
                X[self.discret_cols]) if self.discret_cols else np.array([[] for _ in range(len(X))])
        X = np.concatenate((X_cont, X_cat), axis=1)
        # get categorical arrays
        self.cat_arrays = self.encoder.categories_ if self.discret_cols else []
        self.imutable_idx_list = find_imutable_idx_list(
            imutable_col_names=self.imutable_cols,
            discrete_col_names=self.discret_cols,
            continuous_col_names=self.continous_cols,
            cat_arrays=self.cat_arrays
        )

        # prepare train & test
        train_test_tuple = train_test_split(X, y.to_numpy(), shuffle=False)
        train_X, test_X, train_y, test_y = map(lambda x: x.astype(jnp.float32), train_test_tuple)
        if self.sample_frac:
            train_size = int(len(train_X) * self.sample_frac)
            train_X, train_y = train_X[:train_size], train_y[:train_size]
        self.train_dataset = Dataset(train_X, train_y)
        self.val_dataset = Dataset(test_X, test_y)
        self.test_dataset = self.val_dataset

    def train_dataloader(self, seed, batch_size):
        return DataLoader(self.train_dataset,seed=seed,batch_size=batch_size,
                          pin_memory=True, shuffle=True, num_workers=0)

    def val_dataloader(self, seed, batch_size):
        return DataLoader(self.val_dataset,seed=seed,batch_size=batch_size * 4,
                          pin_memory=True, shuffle=False, num_workers=0)

    def test_dataloader(self, seed, batch_size):
        return DataLoader(self.val_dataset,seed=seed,batch_size=batch_size,
                          pin_memory=True, shuffle=False, num_workers=0)

    def get_sample_X(self, frac: Optional[float]=None):
        if frac is None:
            frac = 0.1
        train_X, train_y = self.train_dataset[:]
        train_size = int(len(train_X) * frac)
        return train_X[:train_size]

In [ ]:
#| include: false
data_configs = {
    "data_dir": "assets/data/s_adult.csv",
    "data_name": "adult",
    "batch_size": 256,
    'sample_frac': 0.1,
    "continous_cols": [
        "age",
        "hours_per_week"
    ],
    "discret_cols": [
        "workclass",
        "education",
        "marital_status",
        "occupation",
        "race",
        "gender"
    ],
}

In [ ]:
dm = TabularDataModule(data_configs)
seed=42
batch_size=data_configs["batch_size"]
t_dataloader = dm.train_dataloader(seed,batch_size)
x, y = next(iter(t_dataloader))
assert x.shape[0] == 256
assert x.shape[1] == 29
assert dm.sample_frac == 0.1

l = 0
t_dataloader = dm.train_dataloader(seed,batch_size)
for i in t_dataloader:
    l += 1
assert l == len(t_dataloader)

t_dataloader = dm.val_dataloader(seed,batch_size)
x, y = next(iter(t_dataloader))
assert x.shape[0] == 256 * 4
assert x.shape[1] == 29



/var/folders/82/4qh59pkn75xdzh4r61851p4h0000gn/T/ipykernel_68202/3988624557.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.data = self.data.astype({col: np.float for col in self.continous_cols})


In [ ]:
dm_2 = TabularDataModule("adult")
seed=42
batch_size=dm_2.batch_size
t_dataloader = dm.train_dataloader(seed,batch_size)
x, y = next(iter(t_dataloader))
print(dm.data)
assert x.shape[0] == 256
assert x.shape[1] == 29
assert dm.sample_frac == 0.1

l = 0
t_dataloader = dm.train_dataloader(seed,batch_size)
for i in t_dataloader:
    l += 1
assert l == len(t_dataloader)

t_dataloader = dm.val_dataloader(seed,batch_size)
x, y = next(iter(t_dataloader))
assert x.shape[0] == 256 * 4
assert x.shape[1] == 29

/Users/chuck/PycharmProjects/cfnet/nbs
        age  hours_per_week      workclass     education marital_status  \
0      42.0            45.0        Private       HS-grad        Married   
1      32.0            40.0  Self-Employed  Some-college        Married   
2      35.0            40.0        Private         Assoc         Single   
3      36.0            40.0        Private       HS-grad         Single   
4      57.0            35.0        Private        School        Married   
...     ...             ...            ...           ...            ...   
32556  66.0            40.0        Private     Bachelors        Married   
32557  35.0            80.0  Self-Employed       HS-grad        Married   
32558  21.0            10.0     Government  Some-college         Single   
32559  24.0            40.0        Private  Some-college        Married   
32560  46.0            40.0        Private       HS-grad        Married   

         occupation   race  gender  income  
0       Blue-Co

/var/folders/82/4qh59pkn75xdzh4r61851p4h0000gn/T/ipykernel_68202/3988624557.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.data = self.data.astype({col: np.float for col in self.continous_cols})
